In [1]:
import requests
import pandas as pd
import os

# URL et clé API
url = "https://www.alphavantage.co/query"
api_key = "O0S7MV4XE2PDKBDF"

# Liste des symboles
symbols = {
    "apple": {"function": "TIME_SERIES_MONTHLY", "symbol": "AAPL"},
    "microsoft": {"function": "TIME_SERIES_MONTHLY", "symbol": "MSFT"},
    "google": {"function": "TIME_SERIES_MONTHLY", "symbol": "GOOGL"},
    "amazon": {"function": "TIME_SERIES_MONTHLY", "symbol": "AMZN"},
    "meta": {"function": "TIME_SERIES_MONTHLY", "symbol": "META"}
}

# Fonction pour télécharger les données
def download_data(asset_name, params):
    params["apikey"] = api_key
    params["datatype"] = "csv"
    response = requests.get(url, params=params)

    if response.status_code == 200:
        filename = f"{asset_name}_data.csv"
        with open(filename, "wb") as file:
            file.write(response.content)
        print(f"Les données de {asset_name} ont été sauvegardées dans {filename}.")
    else:
        print(f"Erreur pour {asset_name}: {response.status_code}, {response.text}")

# Télécharger les données historiques
for asset, params in symbols.items():
    download_data(asset, params)


Les données de apple ont été sauvegardées dans apple_data.csv.
Les données de microsoft ont été sauvegardées dans microsoft_data.csv.
Les données de google ont été sauvegardées dans google_data.csv.
Les données de amazon ont été sauvegardées dans amazon_data.csv.
Les données de meta ont été sauvegardées dans meta_data.csv.


In [2]:
pip install yfinance

Note: you may need to restart the kernel to use updated packages.


In [73]:
import yfinance as yf
import pandas as pd

# Liste des tickers
assets = {
    "Gold": "GC=F",        # Gold Futures
    "Bitcoin": "BTC-USD",  # Bitcoin en USD
    "Ethereum": "ETH-USD", # Ethereum en USD
    "S&P 500": "^GSPC"     # S&P 500 Index
}

# Définir la plage de dates
start_date = "2000-01-01"
end_date = "2024-12-01"

# Télécharger les données pour chaque actif
for asset_name, ticker in assets.items():
    print(f"Téléchargement des données pour {asset_name} ({ticker})...")
    
    # Télécharger les données avec yfinance
    data = yf.download(ticker, start=start_date, end=end_date, interval="1mo")
    
    # Sauvegarder dans un fichier CSV
    filename = f"{asset_name.lower()}_historical_data.csv"
    data.to_csv(filename)
    print(f"Données sauvegardées dans {filename}.")

    # Afficher les premières lignes
    print(data.head())


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Téléchargement des données pour Gold (GC=F)...
Données sauvegardées dans gold_historical_data.csv.
Price            Close        High         Low        Open Volume
Ticker            GC=F        GC=F        GC=F        GC=F   GC=F
Date                                                             
2000-09-01  273.600006  278.399994  269.000000  277.000000   4291
2000-11-01  270.100006  270.200012  263.899994  263.899994  32465
2000-12-01  272.000000  276.000000  268.000000  269.700012   1664
2001-01-01  265.600006  268.399994  262.799988  268.399994  17811
2001-02-01  266.799988  268.899994  255.000000  266.299988   1066
Téléchargement des données pour Bitcoin (BTC-USD)...
Données sauvegardées dans bitcoin_historical_data.csv.
Price            Close        High         Low        Open      Volume
Ticker         BTC-USD     BTC-USD     BTC-USD     BTC-USD     BTC-USD
Date                                                                  
2014-10-01  338.321014  411.697998  289.295990  387.


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Données sauvegardées dans ethereum_historical_data.csv.
Price             Close         High         Low         Open        Volume
Ticker          ETH-USD      ETH-USD     ETH-USD      ETH-USD       ETH-USD
Date                                                                       
2017-12-01   756.732971   881.943970  414.411011   445.209015   79862276544
2018-01-01  1118.310059  1432.880005  742.004028   755.757019  163610220288
2018-02-01   855.198975  1161.349976  574.419006  1119.369995   83393441152
2018-03-01   396.457001   880.302002  368.632996   856.012024   53716179584
2018-04-01   669.924011   708.875000  363.804993   397.253998   60669918656
Téléchargement des données pour S&P 500 (^GSPC)...
Données sauvegardées dans s&p 500_historical_data.csv.
Price             Close         High          Low         Open       Volume
Ticker            ^GSPC        ^GSPC        ^GSPC        ^GSPC        ^GSPC
Date                                                                       
20

In [74]:
# Liste des fichiers CSV avec leurs noms explicites (uniquement pour les entreprises)
companies = [
    "amazon_data.csv",
    "apple_data.csv",
    "google_data.csv",
    "meta_data.csv",
    "microsoft_data.csv"
]

# Créer un DataFrame principal avec toutes les dates possibles de 2000 à 2024
date_range = pd.date_range(start="2000-01", end="2024-12", freq="M")
final_df = pd.DataFrame({'year_month': date_range.to_period('M')})

# Boucle sur chaque fichier CSV
for file in companies:
    # Charger les données
    df = pd.read_csv(file)

    # Récupérer le nom de la société (extrait du nom de fichier)
    company_name = file.replace("_data.csv", "").replace(".csv", "").replace(" ", "")

    # Convertir la colonne timestamp en datetime
    df['timestamp'] = pd.to_datetime(df['timestamp'])

    # Ajouter une colonne année-mois
    df['year_month'] = df['timestamp'].dt.to_period('M')

    # Calculer le prix moyen et garder les colonnes nécessaires
    df = df[['year_month', 'high', 'low', 'volume']]
    df[f'{company_name}_stock_price'] = (df['high'] + df['low']) / 2
    df[f'{company_name}_stock_volume'] = df['volume']

    # Garder uniquement les colonnes de la société traitée
    df = df[['year_month', f'{company_name}_stock_price', f'{company_name}_stock_volume']]

    # Supprimer les colonnes existantes dans final_df avant fusion
    for col in [f'{company_name}_stock_price', f'{company_name}_stock_volume']:
        if col in final_df.columns:
            final_df.drop(columns=col, inplace=True)

    # Fusionner directement avec le DataFrame principal
    final_df = pd.merge(final_df, df, on='year_month', how='left')




/tmp/ipykernel_49256/1865248384.py:11: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  date_range = pd.date_range(start="2000-01", end="2024-12", freq="M")


In [75]:
final_df.head()


,year_month,amazon_stock_price,amazon_stock_volume,apple_stock_price,apple_stock_volume,google_stock_price,google_stock_volume,meta_stock_price,meta_stock_volume,microsoft_stock_price,microsoft_stock_volume
0,2000-01,74.970,262209000,104.000,112099800,NaN,NaN,NaN,NaN,106.745,637437600
1,2000-02,74.500,207457700,108.470,65355200,NaN,NaN,NaN,NaN,99.060,667243800
2,2000-03,67.625,156584600,132.190,77663900,NaN,NaN,NaN,NaN,101.970,1014093800
3,2000-04,54.720,162012500,122.185,77342900,NaN,NaN,NaN,NaN,80.750,1129073300
4,2000-05,51.410,130688600,104.000,87569200,NaN,NaN,NaN,NaN,67.190,672215400


In [77]:
element_files = [
    "bitcoin_historical_data.csv",
    "ethereum_historical_data.csv",
    "gold_historical_data_2000_2024.csv",
    "s&p 500_historical_data.csv"
]
# Boucle sur chaque fichier d'éléments
for file in element_files:
    # Charger les données en ignorant les deux premières lignes inutiles
    df = pd.read_csv(file, skiprows=2)

    # Récupérer le nom de l'élément
    element_name = file.replace("_historical_data.csv", "").replace(".csv", "").replace(" ", "").lower()

    # Renommer les colonnes en utilisant leur position
    df = df.iloc[:, [0, 2, 3, 5]]  # Sélectionne les colonnes Date, High, Low, Volume
    df.columns = ["timestamp", "high", "low", "volume"]  # Renommer pour uniformité

    # Convertir la colonne timestamp en datetime
    df['timestamp'] = pd.to_datetime(df['timestamp'])

    # Ajouter une colonne année-mois
    df['year_month'] = df['timestamp'].dt.to_period('M')

    # Calculer le prix moyen
    df[f'{element_name}_price'] = (df['high'] + df['low']) / 2
    df[f'{element_name}_volume'] = df['volume']

    # Garder uniquement les colonnes nécessaires
    df = df[['year_month', f'{element_name}_price', f'{element_name}_volume']]

    # Supprimer les colonnes existantes avant fusion
    for col in [f'{element_name}_price', f'{element_name}_volume']:
        if col in final_df.columns:
            final_df.drop(columns=col, inplace=True)

    # Fusionner avec le DataFrame principal
    final_df = pd.merge(final_df, df, on='year_month', how='left')


In [78]:
final_df.head()


,year_month,amazon_stock_price,amazon_stock_volume,apple_stock_price,apple_stock_volume,google_stock_price,google_stock_volume,meta_stock_price,meta_stock_volume,microsoft_stock_price,microsoft_stock_volume,bitcoin_price,bitcoin_volume,ethereum_price,ethereum_volume,gold_historical_data_2000_2024_price,gold_historical_data_2000_2024_volume,s&p500_price,s&p500_volume
0,2000-01,74.970,262209000,104.000,112099800,NaN,NaN,NaN,NaN,106.745,637437600,NaN,NaN,NaN,NaN,NaN,NaN,1414.070007,21494400000
1,2000-02,74.500,207457700,108.470,65355200,NaN,NaN,NaN,NaN,99.060,667243800,NaN,NaN,NaN,NaN,NaN,NaN,1384.809998,20912000000
2,2000-03,67.625,156584600,132.190,77663900,NaN,NaN,NaN,NaN,101.970,1014093800,NaN,NaN,NaN,NaN,NaN,NaN,1449.744995,26156200000
3,2000-04,54.720,162012500,122.185,77342900,NaN,NaN,NaN,NaN,80.750,1129073300,NaN,NaN,NaN,NaN,NaN,NaN,1433.294983,20106460000
4,2000-05,51.410,130688600,104.000,87569200,NaN,NaN,NaN,NaN,67.190,672215400,NaN,NaN,NaN,NaN,NaN,NaN,1421.299988,19898300000


In [79]:
print(len(final_df))

299


Valeurs Manquantes

In [80]:
# Compter le nombre total de valeurs manquantes dans final_df
missing_values_count = final_df.isnull().sum().sum()
print(f"Nombre total de valeurs manquantes dans final_df : {missing_values_count}")


Nombre total de valeurs manquantes dans final_df : 1314


VM où il y a au moins une valeur dans une année

In [81]:
# Vérifier les colonnes avec des valeurs numériques uniquement
numeric_columns = final_df.select_dtypes(include=['float64', 'int64']).columns

# Remplacer les valeurs manquantes par la moyenne annuelle si possible
for col in numeric_columns:
    # Calculer la moyenne annuelle pour chaque colonne (en ignorant les NaN)
    annual_mean = final_df.groupby(final_df['year_month'].dt.year)[col].transform('mean')
    
    # Remplacer les NaN par la moyenne annuelle calculée
    final_df[col] = final_df[col].fillna(annual_mean)


In [82]:
missing_values_count = final_df.isnull().sum().sum()
print(f"Nombre total de valeurs manquantes dans final_df : {missing_values_count}")

Nombre total de valeurs manquantes dans final_df : 1150


VM qui restent

In [83]:
for col in numeric_columns:
    # Calculer la moyenne globale (en ignorant les NaN)
    global_mean = final_df[col].mean()
    
    # Remplacer les NaN restants par la moyenne globale
    final_df[col] = final_df[col].fillna(global_mean)

In [84]:
final_df.head()

,year_month,amazon_stock_price,amazon_stock_volume,apple_stock_price,apple_stock_volume,google_stock_price,google_stock_volume,meta_stock_price,meta_stock_volume,microsoft_stock_price,microsoft_stock_volume,bitcoin_price,bitcoin_volume,ethereum_price,ethereum_volume,gold_historical_data_2000_2024_price,gold_historical_data_2000_2024_volume,s&p500_price,s&p500_volume
0,2000-01,74.970,262209000,104.000,112099800,760.629032,1.979314e+08,177.585127,6.236496e+08,106.745,637437600,17705.370867,5.175726e+11,1382.316098,3.549254e+11,270.916667,12806.666667,1414.070007,21494400000
1,2000-02,74.500,207457700,108.470,65355200,760.629032,1.979314e+08,177.585127,6.236496e+08,99.060,667243800,17705.370867,5.175726e+11,1382.316098,3.549254e+11,270.916667,12806.666667,1384.809998,20912000000
2,2000-03,67.625,156584600,132.190,77663900,760.629032,1.979314e+08,177.585127,6.236496e+08,101.970,1014093800,17705.370867,5.175726e+11,1382.316098,3.549254e+11,270.916667,12806.666667,1449.744995,26156200000
3,2000-04,54.720,162012500,122.185,77342900,760.629032,1.979314e+08,177.585127,6.236496e+08,80.750,1129073300,17705.370867,5.175726e+11,1382.316098,3.549254e+11,270.916667,12806.666667,1433.294983,20106460000
4,2000-05,51.410,130688600,104.000,87569200,760.629032,1.979314e+08,177.585127,6.236496e+08,67.190,672215400,17705.370867,5.175726e+11,1382.316098,3.549254e+11,270.916667,12806.666667,1421.299988,19898300000


In [85]:
missing_values_count = final_df.isnull().sum().sum()
print(f"Nombre total de valeurs manquantes dans final_df : {missing_values_count}")

Nombre total de valeurs manquantes dans final_df : 0


In [91]:
# Calculer le maximum des 6 derniers mois
final_df['6_months_max'] = final_df['s&p500_price'].rolling(window=6, min_periods=6).max()

# Calculer la variation en pourcentage par rapport au maximum des 6 derniers mois
final_df['percent_change'] = (final_df['s&p500_price'] - final_df['6_months_max']) / final_df['6_months_max'] * 100

# Détecter les crises avec un seuil de -10 %
final_df['crisis'] = final_df['percent_change'].apply(lambda x: 1 if x <= -10 else (0 if pd.notna(x) else pd.NA))

# Afficher uniquement les lignes où une crise est détectée
crisis_df = final_df[final_df['crisis'] == 1]
print(crisis_df[['year_month', 's&p500_price', '6_months_max', 'percent_change', 'crisis']])

    year_month  s&p500_price  6_months_max  percent_change crisis
11     2000-12   1321.559998   1475.320007      -10.422146      1
13     2001-02   1295.909973   1474.764954      -12.127694      1
14     2001-03   1174.304993   1380.304993      -14.924238      1
15     2001-04   1180.645020   1366.679993      -13.612182      1
20     2001-09   1050.075012   1273.965027      -17.574267      1
21     2001-10   1068.684998   1273.965027      -16.113474      1
22     2001-11   1108.845032   1244.825012      -10.923622      1
29     2002-06   1011.829987   1140.334961      -11.269055      1
30     2002-07    885.070007   1140.334961      -22.385085      1
31     2002-08    899.220001   1140.334961      -21.144222      1
32     2002-09    862.110016   1105.649963      -22.026858      1
33     2002-10    838.035004   1077.774963      -22.243972      1
34     2002-11    906.934998   1011.829987      -10.366859      1
96     2008-01   1370.910034   1532.825012      -10.563174      1
97     200

In [92]:
# Filtrer les lignes où year_month contient 2020 et crisis = 1
filtered_2020_crisis = final_df[(final_df['year_month'].str.startswith('2020')) & (final_df['crisis'] == 1)]

# Afficher les résultats
print(filtered_2020_crisis[['year_month', 's&p500_price', '12_months_max', 'percent_change', 'crisis']])


    year_month  s&p500_price  12_months_max  percent_change crisis
242    2020-03   2664.290039    3276.204956      -18.677553      1
243    2020-04   2701.175049    3276.204956      -17.551707      1
244    2020-05   2917.654907    3276.204956      -10.944066      1


In [95]:
!pip install arch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 978.3/978.3 kB 47.9 MB/s eta 0:00:00


In [96]:
import numpy as np
from arch import arch_model

Test pour Amazon

In [104]:
# Calcul des rendements logarithmiques pour l'or
final_df['gold_return'] = np.log(final_df['gold_historical_data_2000_2024_price'] / final_df['gold_historical_data_2000_2024_price'].shift(1))

# Calcul des rendements logarithmiques pour Amazon
final_df['stock_return'] = np.log(final_df['amazon_stock_price'] / final_df['amazon_stock_price'].shift(1))

# Supprimer les lignes avec des valeurs manquantes (les premières lignes créées par shift)
final_df = final_df.dropna()

# Afficher les premières lignes pour vérifier
print(final_df[['gold_return', 'stock_return']].head())


    gold_return  stock_return
17    -0.018480     -0.052837
18     0.008514     -0.021872
19    -0.001888     -0.285695
20     0.036918     -0.365185
21     0.004938      0.042167


test

In [118]:
import numpy as np
import pandas as pd
from arch import arch_model
from scipy.optimize import minimize
from statsmodels.api import OLS, add_constant

In [124]:
# 1. Préparer les données
final_df = final_df[(final_df['gold_historical_data_2000_2024_price'] > 0) & (final_df['amazon_stock_price'] > 0)]
final_df = final_df.dropna()

# Calculer les rendements logarithmiques
final_df['gold_return'] = np.log(final_df['gold_historical_data_2000_2024_price'] / final_df['gold_historical_data_2000_2024_price'].shift(1)) * 100
final_df['amazon_stock_return'] = np.log(final_df['amazon_stock_price'] / final_df['amazon_stock_price'].shift(1)) * 100
final_df = final_df.dropna()

# 2. Ajuster les modèles GARCH univariés
gold_model = arch_model(final_df['gold_return'], mean='Zero', vol='Garch', p=1, q=1)
gold_fit = gold_model.fit(disp="off")

amazon_model = arch_model(final_df['amazon_stock_return'], mean='Zero', vol='Garch', p=1, q=1)
amazon_fit = amazon_model.fit(disp="off")
# Extraire les résidus standardisés
gold_std_resid = gold_fit.std_resid
amazon_std_resid = amazon_fit.std_resid

# Variables muettes basées sur les résidus standardisés
final_df['dum_gold_neg'] = (gold_std_resid < 0).astype(int)
final_df['dum_stock_neg'] = (amazon_std_resid < 0).astype(int)
final_df['interaction'] = final_df['dum_gold_neg'] * final_df['dum_stock_neg']



In [125]:
# 3. Calcul de la covariance conditionnelle
final_df['covariance_conditionnelle'] = gold_std_resid * amazon_std_resid

# 4. Ajouter la variable de crise
final_df['dum_crise'] = (final_df['crisis'] == 1).astype(int)  # Variable de crise (assurez-vous que 'crisis' existe)

# 5. Régression pour estimer c_GS et delta_GS
X = final_df[['gold_return', 'amazon_stock_return', 'interaction', 'dum_crise']]
X = add_constant(X)  # Ajouter une constante pour inclure c_GS
y = final_df['covariance_conditionnelle']

model_hGS = OLS(y, X)
results_hGS = model_hGS.fit()

# 6. Résultats de la régression
print("\nRésultats de la régression pour h_GS :")
print(results_hGS.summary())


Résultats de la régression pour h_GS :
                                OLS Regression Results                               
Dep. Variable:     covariance_conditionnelle   R-squared:                       0.141
Model:                                   OLS   Adj. R-squared:                  0.128
Method:                        Least Squares   F-statistic:                     10.99
Date:                       Mon, 23 Dec 2024   Prob (F-statistic):           2.91e-08
Time:                               18:46:27   Log-Likelihood:                -337.65
No. Observations:                        272   AIC:                             685.3
Df Residuals:                            267   BIC:                             703.3
Df Model:                                  4                                         
Covariance Type:                   nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
------------

In [126]:
# Extraire les coefficients
c_GS = results_hGS.params['const']
delta_GS = results_hGS.params['dum_crise']
cGS_sum = c_GS + delta_GS
cGS_sum

np.float64(-0.09490447918270542)